<a href="https://colab.research.google.com/github/Beerschtein/Recommended_systems/blob/main/rs_hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/"

 0_97a5c_88fe907_orig.jpg
 app
 BKF
'Colab Notebooks'
 Rec_Sys
 Selfie.xlsx
'БМВ сервисная книжка.pdf'
 Документы
'Копия Яндекс.Практикум. Итоговая практика №2.gsheet'
'Копия Яндекс.Практикум. Урок 1.1. Задание 1–2.gsheet'
'Копия Яндекс.Практикум. Урок 1.1. Задание 3 (1).gsheet'
'Копия Яндекс.Практикум. Урок 1.1. Задание 3.gsheet'
'Копия Яндекс.Практикум. Урок 1.3. Задания 1–3 (1).gsheet'
'Копия Яндекс.Практикум. Урок 1.3. Задания 1–3 (2).gsheet'
'Копия Яндекс.Практикум. Урок 1.3. Задания 1–3.gsheet'
'Новая форма.gform'
'Поверка холодного счетчика.pdf'
'Теория вероятностей'
 Яндекс-практикум


In [ ]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 7.0 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3413737 sha256=6661fc1ce2265b5afd16ba1f6339dbcb5be960e4aac7a19d0fc22b0c3c96a13c
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
data['week_no'].nunique()

95

In [ ]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [ ]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [ ]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [ ]:
item_features = pd.read_csv('./product.csv')
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [ ]:
user_features = pd.read_csv('./hh_demographic.csv')
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [ ]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [ ]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [ ]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head()

CPU times: user 3.91 s, sys: 7.25 ms, total: 3.92 s
Wall time: 3.92 s


In [ ]:
result.head()

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[10283296, 1005186, 837920, 1107466, 914697]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1046825, 995533, 9655098, 911470, 12949538]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1321649, 12811268, 860255, 862660, 65917]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[7157395, 10311300, 6946487, 1119695, 9670430]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[905628, 826201, 637374, 1045031, 1113721]"


In [ ]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [ ]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head()

CPU times: user 125 ms, sys: 1.46 ms, total: 126 ms
Wall time: 127 ms


In [ ]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9798863, 9220223, 12988067, 917216, 15684103]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[892696, 1167633, 9487628, 15719367, 9527482]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1979316, 2524275, 961933, 925694, 929351]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1650889, 6443335, 13842076, 963789, 1094993]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[10182813, 955370, 1710359, 7160764, 1107687]","[6534178, 6533889, 1029743, 6534166, 1082185]"


**Weighted random recommender**

Можно сэмплировать товары случайно, но пропорционально какому-либо весу
Например, прямопропорционально популярности. Вес = log(sales_sum товара)

**ДОМАШНЕЕ ЗАДАНИЕ**

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_probabilities['item_id'])
    probability = np.array(items_probabilities['probability'])
    
    recs = np.random.choice(items, size=n, replace=False, p=probability)



    return recs.tolist()

In [ ]:
%%time

data = data_train
items_probabilities = data.groupby('item_id')['sales_value'].sum().reset_index()

items_probabilities['log'] = np.log1p(items_probabilities['sales_value'])
sum_ = items_probabilities.log.sum()

#probability -- это условная вероятность сэмплирования товара
items_probabilities['probability'] = items_probabilities['log']/sum_.sum()

#сумма по всем товарам д.б. равна единице, проверим
items_probabilities.probability.sum()

CPU times: user 110 ms, sys: 4.62 ms, total: 115 ms
Wall time: 112 ms


In [ ]:
#сумма по всем товарам должна равняться единице, проверим:
items_probabilities.probability.sum()

1.0

In [ ]:
weighted_recs = weighted_random_recommendation(items_probabilities, n=5)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_recs)
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9798863, 9220223, 12988067, 917216, 15684103]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[892696, 1167633, 9487628, 15719367, 9527482]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1979316, 2524275, 961933, 925694, 929351]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1650889, 6443335, 13842076, 963789, 1094993]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[10182813, 955370, 1710359, 7160764, 1107687]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"


In [ ]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [ ]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [ ]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [ ]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head()

In [ ]:
user_item_matrix.shape

(2499, 5001)

In [ ]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [ ]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Fit

In [ ]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.26 s, sys: 9.94 ms, total: 2.27 s
Wall time: 1.65 s


In [ ]:
recs

[(0, 78679.0),
 (3408, 72173.0),
 (2149, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [ ]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 1127831, 1098066]

In [ ]:
result['itemitem'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

In [ ]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9798863, 9220223, 12988067, 917216, 15684103]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[892696, 1167633, 9487628, 15719367, 9527482]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1979316, 2524275, 961933, 925694, 929351]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1650889, 6443335, 13842076, 963789, 1094993]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[10182813, 955370, 1710359, 7160764, 1107687]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]"


 Косинусное сходство и CosineRecommender

In [ ]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.31 s, sys: 23.1 ms, total: 2.34 s
Wall time: 1.76 s


In [ ]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 6666, 981760, 1127831, 1098066]

In [ ]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 73.4 ms, sys: 802 µs, total: 74.2 ms
Wall time: 74.5 ms


In [ ]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9798863, 9220223, 12988067, 917216, 15684103]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 6666, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[892696, 1167633, 9487628, 15719367, 9527482]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 1098066, 981760, 6666, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1979316, 2524275, 961933, 925694, 929351]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 6666, 981760, 1127831, 1098066]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1650889, 6443335, 13842076, 963789, 1094993]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 981760, 6666, 1127831, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[10182813, 955370, 1710359, 7160764, 1107687]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 981760, 6666, 1098066, 826249]"


TF-IDF взвешивание и TFIDFRecommender

In [ ]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.3 s, sys: 20.6 ms, total: 2.32 s
Wall time: 1.69 s


In [ ]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 981760, 1127831, 6666, 1098066]

In [ ]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=False, 
                                    recalculate_user=False)])

CPU times: user 125 ms, sys: 1.74 ms, total: 127 ms
Wall time: 128 ms


In [ ]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9798863, 9220223, 12988067, 917216, 15684103]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[892696, 1167633, 9487628, 15719367, 9527482]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1979316, 2524275, 961933, 925694, 929351]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1650889, 6443335, 13842076, 963789, 1094993]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 981760, 6666, 1127831, 961554]","[1082185, 981760, 1127831, 6666, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[10182813, 955370, 1710359, 7160764, 1107687]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 6666, 1098066, 826249]"


Трюк

In [ ]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.98 s, sys: 10.1 ms, total: 1.99 s
Wall time: 1.41 s


In [ ]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

CPU times: user 56.3 ms, sys: 0 ns, total: 56.3 ms
Wall time: 56.9 ms


In [ ]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9798863, 9220223, 12988067, 917216, 15684103]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[892696, 1167633, 9487628, 15719367, 9527482]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1979316, 2524275, 961933, 925694, 929351]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 878996]","[1082185, 981760, 995242, 1029743, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1650889, 6443335, 13842076, 963789, 1094993]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 981760, 6666, 1127831, 961554]","[1082185, 981760, 1127831, 6666, 961554]","[1082185, 995242, 1029743, 826249, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[10182813, 955370, 1710359, 7160764, 1107687]","[6534178, 6533889, 1029743, 6534166, 1082185]","[83837, 1032114, 822984, 943905, 13161377]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 995242, 1029743, 840361]"


In [ ]:
result.shape

(2042, 8)

**ДОМАШНЕЕ ЗАДАНИЕ**

Измерим качество по precision@5

In [ ]:
import os, sys
    
from metrics import precision_at_k, recall_at_k

In [ ]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0004:random_recommendation
0.1552:popular_recommendation
0.0004:weighted_random_recommendation
0.1329:cosine
0.139:tfidf
0.2199:own_purchases


/content/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


**"random_recommendation" и "weighted_random_recommendation" показали одинаковый результат. Могу предположить, что "взвешивание" мной было выполнено не совсем верным способом и может требоваться доработка.**